# การส่งข้อมูลดิจิทัลด้วยสัญญาณแบบแอนะล็อก

ช่องสัญญาณบางอย่างมีการจำกัดความถี่ต่ำสุดและสูงสุดที่ยอมให้สัญญาณผ่านไปได้ เช่นสายโทรศัพท์ยอมให้สัญญาณเสียงผ่านในช่วง 300-3000 Hz ส่วนช่องสัญญาณสำหรับใช้งานแลนไร้สายมาตรฐานหนึ่งยอมให้สัญญาณวิทยุผ่านในช่วง 2400-2500 MHz เป็นต้น

ในกรณีนี้การส่งข้อมูลโดยใช้สัญญาณแบบดิจิทัล (ซึ่งเรียกว่า baseband transmission) ไม่สามารถทำได้เนื่องจากพลังงานของสัญญาณส่วนใหญ่กระจายตัวอยู่ในบริเวณย่านความถี่ต่ำ จึงต้องมีกระบวนการโยกย้ายความถี่ของสัญญาณให้ไปอยู่ในย่ายที่ต้องการ ซึ่งเรียกว่ากระบวนการมอดูเลชัน (modulation)

## วัตถุประสงค์
* เข้าใจถึงข้อจำกัดของการส่งสัญญาณแบบดิจิทัล (baseband tranmission) ผ่านช่องสัญญาณที่มีการจำกัดความถี่ต่ำสุดและสูงสุด (bandpass channel) 
* รู้จักกระบวนการทำมอดูเลชันแบบต่าง ๆ รวมถึงความต้องการการใช้แบนด์วิธด์ที่แตกต่างกัน
* เข้าใจความหมายของ constellation diagram รวมถึงความสัมพันธ์ระหว่าง bit rate และ baud rate

## การเตรียมตัว
รันคำสั่งในเซลล์ต่อไปนี้เพื่อโหลดไลบรารีและตั้งค่าอื่น ๆ ที่ใช้งานเหมือนที่ผ่านมา

In [ ]:
from sigproc import Signal, start_notebook
bkp = start_notebook()

## โมดูล `analog.py`
ผู้สอนได้เตรียมโมดูล `analog.py` ที่นิยามฟังก์ชันและคลาสที่จำเป็นในการสร้างมอดูเลชันแบบต่าง ๆ เอาไว้ ให้อิมพอร์ททุกอย่างจากโมดูลนี้มารอไว้ใช้งาน

In [ ]:
from analog import *

## กำหนดค่าตัวแปรที่ใช้ร่วมกัน
เซลล์ด้านล่างนิยามตัวแปรเอาไว้ใช้ในการสร้างกลไกมอดูเลชันและข้อมูลที่จะนำมาสร้างเป็นสัญญาณ กลับมาแก้ไขและรันเซลล์นี้ใหม่เพื่อศึกษาพฤติกรรมของสัญญาณที่ถูกสร้างขึ้นด้วยข้อมูลและการตั้งค่าที่แตกต่างกัน

In [ ]:
CARRIER_FREQ = 20   # ความถี่ของคลื่นพาหะ
BIT_RATE = 10       # อัตราส่งข้อมูลเป็นบิตต่อวินาที
DATA = '00011011'   # ข้อมูลดิจิทัลที่ต้องการส่ง

## มอดูเลชันแบบ Binary Amplitude Shift Keying (ASK)
ASK ใช้ค่าแอมพลิจูดของคลื่นพาหะเป็นตัวบ่งชี้ค่าบิตข้อมูลทีละบิต

In [ ]:
mod_ask = ask_modulation(bitrate=BIT_RATE,fc=CARRIER_FREQ)
sig_ask = mod_ask.generate_signal(DATA)
print("data bits =",DATA)
sig_ask.plot_time()
sig_ask.plot_freq()

## มอดูเลชันแบบ On-Off Keying (OOK)
OOK เป็นรูปแบบหนึ่งของ Binary ASK โดยที่ใช้แอมพลิจูดเท่ากับศูนย์แทนบิตข้อมูล 0 

In [ ]:
mod_ook = ook_modulation(bitrate=BIT_RATE,fc=CARRIER_FREQ)
sig_ook = mod_ook.generate_signal(DATA)
print("data bits =",DATA)
sig_ook.plot_time()
sig_ook.plot_freq()

## มอดูเลชันแบบ Binary Phase Shift Keying (PSK)
PSK ใช้ค่าเฟสของคลื่นพาหะเป็นตัวบ่งชี้ค่าบิตข้อมูล โดยใช้รูปแบบคลื่นที่มีเฟสแตกต่างกัน 2 รูปแบบ

In [ ]:
mod_psk = psk_modulation(bitrate=BIT_RATE,fc=CARRIER_FREQ)
sig_psk = mod_psk.generate_signal(DATA)
print("data bits =",DATA)
sig_psk.plot_time()
sig_psk.plot_freq()

## มอดูเลชันแบบ Quadrature Phase Shift Keying (QPSK)
เป็น PSK แบบที่ใช้รูปแบบคลื่นที่เฟสแตกต่างกัน 4 รูปแบบ ทำให้ 1 รูปคลื่นแทนค่าข้อมูลได้ 2 บิต (2 bits/symbol) วงจรสร้างสัญญาณอย่างง่ายอาศัยการสร้างคลื่นซายน์ที่มีมุมแตกต่างกัน $90\circ$ เพื่อนำมาผสมกันให้ได้รูปคลื่นที่มีเฟส 4 รูปแบบคือ $45^\circ$, $135^\circ$, $-45^\circ$ และ $-135^\circ$
![วงจรมอดูเลชันแบบ QPSK](pics/qpsk-circuit.png)

In [ ]:
mod_qpsk = qpsk_modulation(bitrate=BIT_RATE,fc=CARRIER_FREQ)
sig_qpsk = mod_qpsk.generate_signal(DATA)
print("data bits =",DATA)
sig_qpsk.plot_time()
sig_qpsk.plot_freq()

## ผลกระทบที่เกิดจากช่องสัญญาณแบบ band-pass

In [ ]:
sig_tx = sig_ook.copy()
sig_rx = sig_tx.copy()

band = (CARRIER_FREQ-BIT_RATE,CARRIER_FREQ+BIT_RATE)
print("frequency band:",band)
sig_rx.filter(band)

fig = bkp.figure(height=300)
sig_tx.plot_time(fig=fig,line_options=dict(color="green",line_width=2))
sig_rx.plot_time(fig=fig,line_options=dict(color="red",line_width=1))
bkp.show(fig)